# Turning Addresses into Points Using the Census Geocoder

In this Python notebook, I will demonstrate how to read in a csv of [San Diego County private school addresses](http://www.cde.ca.gov/ds/si/ps/), geocode the addresses using Requests and the [Census Geocoding Services API](https://www.census.gov/data/developers/data-sets/Geocoding-services.html) and merge the addresses with the original data source using Pandas.

In [1]:
import pandas as pd
import requests

In [2]:
schools = pd.read_csv('data/schools.csv')
schools.head()

,cds_code,school,street,city,state,zip
0,3.776852e+13,Academia de Talar,31430 Calle de Talar,Bonsall,CA,92003
1,NaN,Academy of Future Men,5444 Mandarin Cove,San Diego,CA,92115
2,3.768339e+13,Academy of Our Lady of Peace,4860 Oregon Street,San Diego,CA,92116
3,NaN,"Accelerated Academics, Inc.","6120 Paseo Del North, Suite L1",Carlsbad,CA,92011
4,3.775615e+13,Agnus Dei Academy,30639 Rabbit Run,Valley Center,CA,92082


The Census Geocoder requires batch files be in the format: unique ID, street, city, state, zip. As such, we need to drop the CDS code and school columns from our dataframe. Because we eventually want to end up with a dataframe that includes those columns, let's create a new dataframe without them instead of just dropping the columns from the original one.

In [3]:
schools_for_geocoding = schools.drop(['cds_code','school'], axis=1)

In [4]:
schools_for_geocoding.head()

,street,city,state,zip
0,31430 Calle de Talar,Bonsall,CA,92003
1,5444 Mandarin Cove,San Diego,CA,92115
2,4860 Oregon Street,San Diego,CA,92116
3,"6120 Paseo Del North, Suite L1",Carlsbad,CA,92011
4,30639 Rabbit Run,Valley Center,CA,92082


Now we can export the dataframe as a csv for geocoding. By default, pandas saves the index as a column at the start of the csv. As such, we might as well use the index as the unique ID column the Census Geocoder requires. The Geocoder will also attempt to geocode the header row so let's leave that out.

In [5]:
schools_for_geocoding.to_csv('schools_for_geocoding.csv', header=False)

Now we will use Requests to call the Census Geocoding API, specifying the options we want (benchmark, vintage, etc.) and passing the csv file as part of the request. You can check out the Census Geocoding Services API documentation for all the available options.

In [6]:
url = 'https://geocoding.geo.census.gov/geocoder/geographies/addressbatch'
payload = {'benchmark':'Public_AR_Current','vintage':'Current_Current'}
files = {'addressFile': ('schools_for_geocoding.csv', open('schools_for_geocoding.csv', 'rb'), 'text/csv')}
r = requests.post(url, files=files, data = payload)
print (r.text)

"35","9028 Westmore Road, San Diego, CA, 92126","Match","Exact","9028 WESTMORE RD, SAN DIEGO, CA, 92126","-117.13123,32.916565","195318950","L","06","073","008357","1008"
"36","484 Vista Place, Chula Vista, CA, 91910","Match","Exact","484 VISTA PL, CHULA VISTA, CA, 91910","-117.06671,32.637478","195356871","R","06","073","012800","1003"
"33","1114 Ninth  Avenue, Coronado, CA, 92118","Match","Non_Exact","1114 9TH ST, CORONADO, CA, 92118","-117.17879,32.687214","636006778","R","06","073","010900","2026"
"159","4786 Caminito Borrego, San Diego, CA, 92130","No_Match"
"34","7929 La Mesa Boulevard, La Mesa, CA, 91942","Match","Exact","7929 LA MESA BLVD, LA MESA, CA, 91942","-117.02656,32.764984","195359317","R","06","073","014601","1018"
"158","3002 Armstrong Street, San Diego, CA, 92111","Match","Exact","3002 ARMSTRONG ST, SAN DIEGO, CA, 92111","-117.165405,32.80045","195714248","L","06","073","008701","1001"
"39","2081 Bear Valley Parkway, Suite B220, Escondido, CA, 92027","Match","Exact",

Save the json response as a csv file.

In [7]:
with open('geocoded_addresses.csv', 'w') as outfile:
    outfile.writelines(r.text)
outfile.close()

If you inspect the file of geocoded addresses, you'll see there are lots of potentially useful geographic features returned. However, we don't need all these columns for our purposes here.  
  
We really only want the ID to merge the dataframe to the original schools dataframe, the input address to double check that the merge worked correctly, the match type and matched address to confirm the input address was correctly geocoded and, of course, the latitude and longitude. We'll also have to specify names for the columns we will be importing since the csv file lacks a header row.

In [8]:
geocoded_addresses = pd.read_csv('geocoded_addresses.csv', usecols=[0,1,3,4,5,6], 
                               names=['id','input_address','match','match_type','matched_address','latitude','longitude','side','state','county','tract','block']
                              )
geocoded_addresses.head()

,id,input_address,match_type,matched_address,latitude,longitude
0,35,"9028 Westmore Road, San Diego, CA, 92126",Exact,"9028 WESTMORE RD, SAN DIEGO, CA, 92126","-117.13123,32.916565",195318950.0
1,36,"484 Vista Place, Chula Vista, CA, 91910",Exact,"484 VISTA PL, CHULA VISTA, CA, 91910","-117.06671,32.637478",195356871.0
2,33,"1114 Ninth Avenue, Coronado, CA, 92118",Non_Exact,"1114 9TH ST, CORONADO, CA, 92118","-117.17879,32.687214",636006778.0
3,159,"4786 Caminito Borrego, San Diego, CA, 92130",NaN,NaN,NaN,NaN
4,34,"7929 La Mesa Boulevard, La Mesa, CA, 91942",Exact,"7929 LA MESA BLVD, LA MESA, CA, 91942","-117.02656,32.764984",195359317.0


And now we can merge the dataframe of geocoded addresses with the original schools dataframe that contains the CDS code and name for each school.

In [9]:
geocoded_schools = pd.merge(schools, geocoded_addresses, left_index=True, right_on='id')
geocoded_schools.drop('id', axis=1, inplace=True) # Drop the id column as we no longer need it post-merge
geocoded_schools.head()

,cds_code,school,street,city,state,zip,input_address,match_type,matched_address,latitude,longitude
51,3.776852e+13,Academia de Talar,31430 Calle de Talar,Bonsall,CA,92003,"31430 Calle de Talar, Bonsall, CA, 92003",Non_Exact,"31430 CLL DE TALAR, BONSALL, CA, 92003","-117.168106,33.28461",195742062.0
49,NaN,Academy of Future Men,5444 Mandarin Cove,San Diego,CA,92115,"5444 Mandarin Cove, San Diego, CA, 92115",Exact,"5444 MANDARIN COVE, SAN DIEGO, CA, 92115","-117.07778,32.75263",612897107.0
46,3.768339e+13,Academy of Our Lady of Peace,4860 Oregon Street,San Diego,CA,92116,"4860 Oregon Street, San Diego, CA, 92116",Exact,"4860 OREGON ST, SAN DIEGO, CA, 92116","-117.135155,32.765354",195345213.0
44,NaN,"Accelerated Academics, Inc.","6120 Paseo Del North, Suite L1",Carlsbad,CA,92011,"6120 Paseo Del North, Suite L1, Carlsbad, CA, ...",NaN,NaN,NaN,NaN
59,3.775615e+13,Agnus Dei Academy,30639 Rabbit Run,Valley Center,CA,92082,"30639 Rabbit Run, Valley Center, CA, 92082",Exact,"30639 RABBIT RUN, VALLEY CENTER, CA, 92082","-117.00871,33.263508",195385805.0


And there you have it: a dataframe of geocoded private schools in San Diego County.